In [1]:
%load_ext Cython

# Compilation

Without C libraries: put codeblocks starting with `%%cython` into a file `cythoncode.pyx`
create `setup.py` with code:
```python
from distutils.core import setup
from Cython.Build import cythonize
setup( name="cythoncode", ext_modules = cythonize("cythoncode.pyx") )
```
run `python setup.py build_ext --inplace`;

in main program, `from cythoncode import *`

With `cimport numpy`:
```python
import os
import numpy
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

setup(cmdclass = {'build_ext': build_ext},
      name="cythoncode",
      ext_modules=[Extension("cythoncode",
                             sources=["cythoncode.pyx"])],
      include_dirs=[numpy.get_include(),
                    os.path.join(numpy.get_include(), 'numpy')])
```

With custom C compilation flags:
```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

setup(  name = "cythoncode",
        cmdclass = {"build_ext": build_ext},
        ext_modules = [ Extension("cythoncode",
                                  sources=["cythoncode.pyx"],
                                  libraries=["m"],  #for using C's math lib
                                  extra_compile_args = ["-ffast-math"])])
```

# using C/C++ libraries

for standard [C libraries](https://github.com/cython/cython/tree/master/Cython/Includes/libc) or [C++ libraries](https://github.com/cython/cython/tree/master/Cython/Includes/libcpp):
```python 
from libc.math cimport sqrt```

For custom library (in header file `custom.h`):
```cython
cdef extern from "custom.h":
    double sin(double x)
```
To allow it to be used by other python codes, add `cpdef` before `double sin`

# Using [Numpy](http://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html)

```cython
cimport numpy as np```

# Timing

In [2]:
def fib(n):
    if n <= 1: return n
    else: return fib(n-1) + fib(n-2)

In [3]:
%%cython
def fibc(n):
    if n <= 1: return n
    else: return fibc(n-1) + fibc(n-2)

In [4]:
%timeit fib(12)
%timeit fibc(12)

36.9 µs ± 594 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
12.5 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# `cdef` local variables

In [14]:
%%cython
def fibx(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

# `cython` function that works even if `gcc` compilation fails

In [ ]:
%%cython
import cython
@cython.locals(n=cython.int)
def fib_pure_python(n):
    cython.declare(a=cython.int, b=cython.int, i=cython.int)
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

# Pairwise Distances (numpy list)

In [5]:
import numpy as np
X = np.random.random((500, 3))

In [6]:
def pairwise_v1(X):
    X = np.asarray(X)    
    n_samples, n_dim = X.shape
    D = np.empty((n_samples, n_samples))

    for i in range(n_samples):
        for j in range(n_samples):
            D[i, j] = np.sqrt(np.sum((X[i] - X[j]) ** 2))
    return D

In [7]:
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
cimport cython

#no bounds check for array; disable negative (count from end) indexing
@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_v2(np.ndarray[double, ndim=2, mode='c'] X not None): #mode='c' indicates C-ordered (contiguous in memory)
    
    cdef np.intp_t i, j, n_samples, N # np.intp: Integer used for indexing (unsigned). _t means the type of it
    cdef double tmp, d    
    n_samples = X.shape[0]
    N = X.shape[1]

    cdef np.ndarray[double, ndim=2, mode='c'] D = np.empty((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            d = 0
            for k in range(N):
                tmp = X[i,k] - X[j,k]
                d += tmp * tmp
            D[i, j] = sqrt(d)

    return D

[Typed Memoryview](http://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html) - faster passing arrays as arguments

In [8]:
%%cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_v3(double[:, ::1] X not None): # ::1 means 2nd dim inc one element apart in memory. can be replaced by : since np array is of this form
    
    cdef np.intp_t i, j, n_samples, N
    cdef double tmp, d    
    n_samples = X.shape[0]
    N = X.shape[1]

    cdef double[:, ::1] D = np.empty((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            d = 0
            for k in range(N):
                tmp = X[i,k] - X[j,k]
                d += tmp * tmp
            D[i, j] = sqrt(d)

    return np.asarray(D)  #convert from memory view back to ordinary numpy array

In [9]:
%timeit pairwise_v1(X)
%timeit pairwise_v2(X)
%timeit pairwise_v3(X)

1.46 s ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
667 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
618 µs ± 5.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
